# Project Title: 
Telecommunications Customer Churn Prediction Analysis

# Business Understanding

## 1. Introduction
This project aims to assist a telecom company in understanding their data and predicting customer churn. The company has provided access to three different datasets: the first dataset with 3000 records stored in a remote Microsoft SQL Server database, the second dataset with 2000 records stored on OneDrive, and the third dataset hosted on a GitHub repository.

### 1.1. Objectives
Understand the data: Our first objective is to provide insights into the telecom company's data, including customer demographics, services availed, and payment details. This understanding will enable the company to make informed business decisions.

Find the lifetime value of each customer: By analyzing the data, we aim to identify factors that influence the rate at which customers churn. Understanding customer behavior and identifying key predictors will help the telecom company estimate the lifetime value of each customer.

Predict customer churn: The primary objective is to develop a predictive model that accurately determines whether a customer is likely to churn or not. We will employ machine learning algorithms such as logistic regression, decision trees, support vector machines, random forest, etc., to build a model that effectively predicts customer churn.

### 1.2. Methodology
To achieve our objectives, we will follow the CRISP-DM framework, which consists of the following steps:

Data exploration: We will thoroughly explore the datasets to gain insights into the available variables, their distributions, and relationships. This step will provide us with an initial understanding of the data and help identify any data quality issues.

Missing value computations: We will identify missing values in the datasets and decide on an appropriate strategy for handling them. This may involve imputing missing values or removing data points with missing values.

Feature engineering: We will perform feature engineering to transform and create new variables that can potentially improve the predictive power of our models. This step may include encoding categorical variables, scaling numerical variables, or creating interaction terms.

Model development: We will utilize various machine learning algorithms such as logistic regression, decision trees, support vector machines, random forest, etc., to develop predictive models for customer churn. We will train the models on a subset of the data and evaluate their performance using appropriate metrics.

Model evaluation and interpretation: We will evaluate the trained models using evaluation metrics such as accuracy, precision, recall, and F1-score. Additionally, we will interpret the models to understand the factors driving customer churn and their relative importance.

Model optimization and hyperparameter tuning: We will fine-tune the models by optimizing their hyperparameters to improve their performance. This step may involve techniques like grid search or random search to find the optimal combination of hyperparameters.

By following this methodology, we aim to provide valuable insights to the telecom company and develop a reliable predictive model for customer churn.

# Load Datasets

## Installations

In [1]:
%pip install pyodbc
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Load first dataset from SQL database

In [ ]:
# Import necessary libraries
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Establish a connection
server = 'dap-projects-database.database.windows.net'
database = 'dapDB'
username = 'dataAnalyst_LP2'
password = 'A3g@3kR$2y'

# Create the connection string using the ODBC driver format
conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Establish the connection using the connection string
conn = pyodbc.connect(conn_str)

In [ ]:
# Query the database to retrieve the data
query = 'SELECT TOP 3000 * FROM LP2_Telco_churn_first_3000'
df_db = pd.read_sql(query, conn)

In [ ]:
# Close connection
conn.close()

In [ ]:
# Display the dataframe
df_db.head()

## Load second dataset (excelfile)

In [ ]:
# Read the excel file into a pandas dataframe
df_excel = pd.read_excel('Telco-churn-second-2000.xlsx')

# Display the dataframe
df_excel.head()

## Load third dataset (csv file)

In [ ]:
# Read third dataset
df_csv = pd.read_csv('LP2_Telco-churn-last-2000.csv')

# Display the dataframe
df_csv.head()

# Questions and Hypothesis

## Hypothesis

H0: The churn rate of customers in the telecom company is not significantly influenced by various factors related to their 
demographics, services, and payment methods.

H1: The churn rate of customers in the telecom company is influenced by various factors related to their demographics, services, and payment methods.

## Questions
1. Is there a relationship between the monthly charges and the likelihood of churn?
2. Do customers who have internet service, specifically fiber optic, exhibit a higher churn rate compared to those with DSL or no internet service?
3. Does the availability of online security, online backup, device protection, and tech support impact the churn rate?
4. How does the churn rate vary based on the customers' gender?
5. Does the presence of a partner influence the likelihood of churn?
6. Is there a correlation between the tenure of customers and their churn rate?

# Data Exploration

## Explore individual datasets 

### Explore The Dataframe from the SQL Database(df_db)

In [ ]:
# Column information
df_db.info()

In [ ]:
df_db.shape

In [ ]:
# check unique values of each column
for column in df_db.columns:
    print('Column: {} - Unique Values: {}'.format(column, df_db[column].unique()))
    print('==='*15)

In [ ]:
# Summary statistics of df_db to get insights into the distribution and basic characteristics of the numerical variables
df_db.describe()

In [ ]:
# Handling missing values
df_db.isnull().sum()

### Explore df_excel

In [ ]:
# Call the dataset df_excel
df_excel.head()

In [ ]:
# Column information
df_excel.info()

In [ ]:
# check unique values of each column
for column in df_excel.columns:
    print('Column: {} - Unique Values: {}'.format(column, df_excel[column].unique()))
    print('==='*15)

In [ ]:
# Summary statistics of df_excel to get insights into the distribution and basic characteristics of the numerical variables
df_excel.describe()

In [ ]:
# Handling missing values
df_excel.isnull().sum()

### Explore df_csv

In [ ]:
# Call the dataset df_csv
df_csv.head()

In [ ]:
# Column information
df_csv.info()

In [ ]:
# check unique values of each column
for column in df_csv.columns:
    print('Column: {} - Unique Values: {}'.format(column, df_csv[column].unique()))
    print('==='*15)

In [ ]:
# Summary statistics of df_csv to get insights into the distribution and basic characteristics of the numerical variables
df_csv.describe()

In [ ]:
# Handling missing values
df_csv.isnull().sum()